In [21]:
!pip install OWSLib

    100% |████████████████████████████████| 163kB 694kB/s ta 0:00:01
    100% |████████████████████████████████| 11.2MB 41kB/s  eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d3/e3/d9f046b5d1c94a3aeab15f1f867aa414f8ee9d196fae6865f1d6a0ee1a0b/pytz-2021.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/61/08076519c80041bc0ffa1a8af0cbd3bf3e2b62af10435d269a9d0f40564d/requests-2.27.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 122kB 1.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 61kB 1.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/37/45/946c02767aabb873146011e6657

## Testing API access to S-122 data through an OGC API.

In [37]:
# something on OGC API
from owslib.ogcapi.features import Features


## Access to OGC API via OWS lib interfaces. 
List out all the collections...

In [38]:
w = Features('http://18.130.213.16/ogcfmsdi/')
feature_collections = w.feature_collections()
print(feature_collections)

['S-100Vector', 'S-122WDPA', 'S-122Dev', 'S-122JNCC', 'S-122DNK', 'S-122DEU', 'TEST']


In [39]:
denmark = w.collection_items('S-122DNK')

## Complex attributes
they're not strings, they're JSON dictionaries, you can filter on these though, but you have to escape all the 's and get the formatting right, it's not ideal but it is possible to make up a search string on just the name (if you know the language). What I'm not sure is whether you can do a more general search. pygeoapi only supports simple profile of CQL though which hampers things somewhat.

In [40]:
print(denmark['features'][0]['properties']['featureName']['name'])

Nysø, Bøndernes Egehoved


In [45]:
print(denmark['features'][3]['properties']['featureName']['name'])
print(denmark['features'][3]['properties']['featureName'])
d4 = w.collection_items('S-122DNK', featureName='{\'name\': \'Ertholmene\', \'language\': \'DNK\'}')
print(d4['numberMatched'])


Ertholmene
{'name': 'Ertholmene', 'language': 'DNK'}
3


## How to retrieve features by relationship

Each relationship can be included as a distinct link (although we may have to group these by relationship name as there is a multiplicity of relationships between features and information types). so, it is possible to grab features, examine the inter-feature links and retrieve attribution from those links. This example retrieves the associated Authority feature and looks for the categoryOfAuthority it contains.

In [42]:
denmark2 = w.collection_items('TEST')
print(denmark2['features'][0]['id'])
print(denmark2['features'][0]['links'])
print( (denmark2['features'][0]['links']).keys() )
relatedlink = denmark2['features'][0]['links']['protectedAreaAuthority']
print('->' + relatedlink)
d3 = w.collection_item('TEST',str(relatedlink))
print(d3['properties']['categoryOfAuthority'])


ID0
{'protectedAreaAuthority': 'ID1'}
dict_keys(['protectedAreaAuthority'])
->ID1
environmental


In [43]:
jncc1 = w.collection_items('S-122JNCC')
print(jncc1['features'][0])
relatedlink = jncc1['features'][1]['links']['protectedAreaAuthority']
print('->' + relatedlink)
d3 = w.collection_item('S-122JNCC',str(relatedlink))
print(d3['properties']['featureName']['name'])

{'type': 'Feature', 'id': 'JNCC', 'properties': {'featureType': 'Authority', 'categoryOfAuthority': 'environmental', 'featureName': {'name': 'JNCC'}}}
->JNCC
JNCC


## Filtering.
This retrieves features according to the IUCN category. Note that the category is not conformant with the S-122 enumeration values (yet). It would be possible to read in a JSON version of the feature catalogue and establish what these are (next step in this notebook).

In [44]:
denmark4 = w.collection_items('S-122DNK', categoryOfMarineProtectedArea='IV')
print(denmark4['numberMatched'])
print(denmark4['features'][1]['properties']['featureName'])

germany4 = w.collection_items('S-122DEU', categoryOfMarineProtectedArea='IV')
print(germany4['numberMatched'])
print(germany4['features'][1]['properties']['featureName'])

26
{'name': 'Ulvshale', 'language': 'DNK'}
66
{'name': 'Insel Vilm', 'language': 'DEU'}
